In [47]:
import math
from Crypto.Util import number
import copy
import random
import time 

#divide et impera method
def evaluate_Si(primes:list, a:int, n:int) -> list: 
    num = len(primes)
    if num == 1:
        return [a]

    a1 = []
    a2 = []

    partial_a1 = a
    partial_a2 = a

    #for each primes number append a^primes[i] mod n
    for i in range(math.floor(len(primes)/2)):
        partial_a2 = pow(partial_a2, primes[i], n)
        a1.append(primes[i])
    #for each primes number append a^primes[i] mod n
    for i in range(math.floor(len(primes)/2), len(primes)):
        partial_a1 = pow(partial_a1, primes[i], n)
        a2.append(primes[i])

    #recursive call to self fun
    l1 = evaluate_Si(a1, partial_a1, n)
    l2 = evaluate_Si(a2, partial_a2, n)

    si_list = []
    for element in l1:
        si_list.append(element)
    for element in l2:
        si_list.append(element)

    return si_list

def keyGen(k:int,qlen:int, l:int) -> tuple:
    #generation of two k/2-bit primes p and q
    p =number.getPrime(math.floor(k/2))
    q = number.getPrime(math.floor(k/2))
    #set n = p*q (rsa)
    n = p*q
    #fix a=5 random value
    a = 5


    
    #list of l+1-bit q random primes
    random_primes = []
    while True:
        #temporary prime
        tmp = number.getPrime(l + 1)
        #boolean variable to check if prime number is already used
        alreadyUsed = False
        #foreach to check duplicate
        for num in random_primes:
            if tmp == num:
                alreadyUsed = True
        if alreadyUsed:
            continue
        #add prime number if is not duplicated
        random_primes.append(tmp)
        #check if are generated q primes 
        if len(random_primes) == qlen:
            break
    
    #call fun to evaluate Si element
    si_list = evaluate_Si(random_primes, a, n)
    return n, random_primes, a, si_list




n, ei_list, a, si_list = keyGen(k=1024,qlen=100,l=25)
print("Done!")




Done!


In [48]:
#commit fun
def commit(messages:list, S:list, n:int) -> int:
    #set neutral element c = 1
    c = 1
    for i in range(len(messages)):
        #set c = s1^m1*...*sq^mq
        c = (c*pow(S[i], messages[i], n))%n
    return c

dim = 100
messages = [random.randint(0,1000000) for i in range(dim)]
c = commit(messages, si_list, n)
print(c)

52866892369391581023216145941619339315938013723135853268558914640048034122625334086258218379834938997033241163006511408793884724478520543764155513251137010567826149674447878578176223144530573682921673397059193261481875057014990011297146493227589285385141124414138012678867473916975814773885514595999111383977


In [49]:
#opening fun
def open(messages:list, ei_list:list, a:int, n:int, i:int) -> int:

    #creating deepcopied list of ei_list primes
    newlist = copy.deepcopy(ei_list)
    #remove ei-th primes from list
    newlist.remove(ei_list[i])
    #recalculation of Si elements without ei-th prime number
    newsi_list = evaluate_Si( newlist, a, n)
    #set prod to neutral element 1
    prod = 1

    counter = 0
    for j in range(len(newsi_list)): 
        #if ei-th random primes is at index i, skip (one S value is gone)
        if(counter == i):
            counter += 1
        #evaluate opening value
        prod = (prod*pow(newsi_list[j], messages[counter], n))%n
        #next message pointer
        counter += 1

    return prod

proof = open(messages, ei_list, a, n, 0)
#later use (opening at pos 1)
oldProof = open(messages, ei_list, a, n, 1)
print(proof)

70339413330758113348367419250336454817051897158076461049277128955015430904853387251314298255472352630113917805949695470897870589068107411953104896389318374194807228909168034579972293704612932728356975695087942340131465702071318699033114221442705416738023428586111805117890396323692238548883143776895917168615


In [50]:
#verify fun
def verify(C:int, message:int, i:int, open:int, S:list, ei_list:list, n:int) -> bool:

    #evaluate proof^ei-th % n
    proofToEi =pow(open, ei_list[i], n)
    #evaluate si-th^message * (proofToEi)
    newC = proofToEi*pow(S[i], message, n)%n

    #returns 1 if two commitment are equals, else 0
    if(C == newC):
        return 1
    else:
        return 0

#start_time = time.time()
verified = verify(c, messages[0], 0, proof, si_list, ei_list, n)
#print("Verify proof required: " + str(time.time()-start_time) + " seconds")
if(verified):
    print("Verifying worked")
else:
    print("Verifying not worked")

Verifying worked


In [51]:
#comute commitment update
def update(C: int, m:int, new_m:int,i:int,si_list:list,n:int) -> int:
    new_c = 1
    b = pow(si_list[i],(new_m-m),n)
    new_c = (C * b) % n
    return new_c

newmessage = random.randint(1,12234567) #random new message, update position 0
    #start_time = time.time()
newC = update(c,messages[0],newmessage,0,si_list,n)
    #print("Update commitment required: " + str(time.time()-start_time) + " seconds")

print(newC)
verified2 = verify(newC, newmessage, 0, proof, si_list, ei_list, n)
if(verified2):
    print("Verifying update worked")
else:
    print("Verifying update not worked")

23854363515748878483795633149738348986322621356197138322812913891271647175011575937197231942479724431853370530713889558852034874595713822315629343725683205203179392128122158998598237757473716073370450723474844061376104098056716995835312084932932939847009617440047626125299124039200470523331326367208143198455
Verifying update worked


In [52]:
def updateProof(oldProof: int, e:list, oldm: int, newm: int, a:int, i:int, j:int, n:int) -> int:
    alt = a
    for k in range(len(e)):
        if(k == i or k == j):
            continue
        alt = pow(alt, e[k], n)

    rootS = alt
    right = pow(rootS, newm - oldm, n)
    full = (oldProof*right)%n
    return full

#update proof at pos j (newC is the updated commitment)
newProof = updateProof(oldProof, ei_list, messages[0], newmessage, a, 0, 1, n)
print(newProof)
verified3 = verify(newC, messages[1], 1, newProof, si_list, ei_list, n)
if(verified3):
    print("Verifying update proof worked")
else:
    print("Verifying update proof not worked")

#sanity check for previous proof

verified4 = verify(newC,messages[1],1,oldProof,si_list,ei_list,n)

if(verified4):
    print("Old proof still works")
else:
    print("Old proof no longer works")

33055366032130209519242089043106001451978491440942366661304682212259181470985511245030998548537409620244664643480599272774242412849882910215296843538272008433572335718344062651360626423878963247309103888458347401405477500538833490648965634357167972046498832077469012584746420569032832215605424936338786962948
Verifying update proof worked
Old proof no longer works
